In [ ]:
import matplotlib.pyplot as plt
import requests
import numpy as np
import pandas as pd
import torch
import torchvision
import torchaudio
import os
from os.path import isfile, join, isdir, exists
import numpy as np
import matplotlib.pyplot as plt
from speechbrain.pretrained import SepformerSeparation as separator

import soundfile as sf

# torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the data

directory = "standardized_split_audio"
save_directory = "audio_split_sb"
if not exists(save_directory):
   os.mkdir("./"+save_directory)
    
dirnames = [f for f in os.listdir(directory) if isdir(join(directory, f))]

for d in dirnames:
    if not exists(join(save_directory, d)):
        os.mkdir(join(save_directory, d))

tbc = [2, 12, 16, 17, 27, 29, 36, 53, 62, 76, 83, 93, 95, 110, 111, 112, 122, 38, 71, 84, 85, 100, 117]

fnames = []
for dirs in dirnames:
    for f in os.listdir(join(directory, dirs)):
        if int(f[:-5]) in tbc:
            fnames.append(join(dirs, f))

for fname in fnames:
    f = fname
    print("starting ", fname)
    
    fname = join(directory, fname)
    if(fname[-3:] != 'wav'):
        continue
    
    # model = BaseModel.from_pretrained('JorisCos/ConvTasNet_Libri2Mix_sepnoisy_16k', sample_rate=44100)
    model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr', run_opts={"device": device})

    est_sources = model.separate_file(fname)
    est_sources = est_sources.cpu()
    transform = torchaudio.transforms.Resample(44100, 8000)
    est_sources = transform(est_sources)
    torchaudio.save(join(save_directory, f[:-4] + "_0.wav"), est_sources[:, :, 0], 8000)
    torchaudio.save(join(save_directory, f[:-4] + "_1.wav"), est_sources[:, :, 1], 8000)
    
    # model.separate(fname, save_dir=save_directory, force_overwrite=True)
    print("Done with ", f)